In [15]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

# Simulação de série temporal
np.random.seed(42)
N = 1000
df = pd.DataFrame({'Y': np.random.randn(N).cumsum()})

# Indicador: média móvel
df['X1'] = df['Y'].rolling(window=3).mean().shift(1)

# Lags
df['Y_lag1'] = df['Y'].shift(1)
df['Y_lag2'] = df['Y'].shift(2)

df.dropna(inplace=True)

# Split em treino + último registro para previsão futura
train = df.iloc[:-1]
last_row = df.iloc[-1:]

X_train = train[['X1', 'Y_lag1', 'Y_lag2']]
y_train = train['Y']

# Treinamento
model = XGBRegressor()
model.fit(X_train, y_train)

df.head()
pd.concat([X_train, y_train], axis=1)

,X1,Y_lag1,Y_lag2,Y
3,0.620434,1.006138,0.358450,2.529168
4,1.297919,2.529168,1.006138,2.295015
5,1.943441,2.295015,2.529168,2.060878
6,2.295020,2.060878,2.295015,3.640091
7,2.665328,3.640091,2.060878,4.407525
...,...,...,...,...
994,18.711892,18.174843,18.856827,17.173223
995,18.068298,17.173223,18.174843,16.892123
996,17.413396,16.892123,17.173223,18.689809
997,17.585052,18.689809,16.892123,19.330652


In [16]:
# Forecast 3 steps ahead (iterativo):
forecast_steps = 3
predictions = []

current_Y_lag1 = last_row['Y'].values[0]
current_Y_lag2 = last_row['Y_lag1'].values[0]
current_X1 = last_row['X1'].values[0]

for step in range(forecast_steps):
    # Monta X para próximo passo
    X_next = pd.DataFrame({
        'X1': [current_X1],
        'Y_lag1': [current_Y_lag1],
        'Y_lag2': [current_Y_lag2]
    })
    display(X_next)
    
    # Faz previsão
    y_pred = model.predict(X_next)[0]
    predictions.append(y_pred)
    
    # Atualiza lags para próxima iteração
    current_Y_lag2 = current_Y_lag1
    current_Y_lag1 = y_pred
    current_X1 = np.mean([y_pred, current_Y_lag1, current_Y_lag2])  # Exemplo simples de atualização de média móvel "na mão"

print(predictions)


,X1,Y_lag1,Y_lag2
0,18.926645,19.332056,18.759473


,X1,Y_lag1,Y_lag2
0,18.725618,18.4224,19.332056


,X1,Y_lag1,Y_lag2
0,19.543022,20.103334,18.4224


[18.4224, 20.103334, 20.534325]


# Estudo de avaliação do tempo de execução de cada backtest-estrategia

In [1]:
import pandas as pd

In [ ]:
NOME_MOEDA = 'BTC'
CANDLE_PERIOD = '15m'
DEFAULT_START_DATE='01/03/2025 00:00'
DEFAULT_END_DATE='31/03/2025 07:45'
STOCK_CODE = NOME_MOEDA  # Código da Criptomoeda
OPERATION_CODE = NOME_MOEDA + "USDT"  # Código da operação (cripto + moeda)
INITIAL_BALANCE = 1000  # Valor de investimento inicial em USDT ou BRL

from tests.baixar_candles import baixar_candles
dados_candles = baixar_candles(OPERATION_CODE, DEFAULT_START_DATE, DEFAULT_END_DATE, CANDLE_PERIOD, ajuste=True)
dados_candles.shape[0] - 120

2912

In [91]:
fd = pd.DataFrame(columns=['dt_inicio', 'dt_end', 'qtde'])

for i in range(1, 9):
    # i = 1
    from datetime import datetime, timedelta
    import pytz

    def str_para_datetime(data_str):
        dt = datetime.strptime(data_str, "%d/%m/%Y %H:%M")
        return pytz.timezone("America/Sao_Paulo").localize(dt)

    dt_inicio = str_para_datetime(DEFAULT_START_DATE)
    # print(dt_inicio)

    dt_end = str_para_datetime(DEFAULT_START_DATE) + (timedelta(minutes=15) * 100 * i)
    # print(dt_end)

    df_filter = dados_candles[(dados_candles['open_time'] >= dt_inicio) & (dados_candles['open_time'] < dt_end)]
    # print(len(df_filter))

    # display(pd.concat([df_filter.head(), df_filter.tail()], axis=0))

    # print(df_filter['open_time'][df_filter.index[0]])
    # print(df_filter['open_time'][df_filter.index[-1]])

    fd.loc[i-1] = [dt_inicio, dt_end, len(df_filter)]
fd

,dt_inicio,dt_end,qtde
0,2025-03-01 00:00:00-03:00,2025-03-02 01:00:00-03:00,100
1,2025-03-01 00:00:00-03:00,2025-03-03 02:00:00-03:00,200
2,2025-03-01 00:00:00-03:00,2025-03-04 03:00:00-03:00,300
3,2025-03-01 00:00:00-03:00,2025-03-05 04:00:00-03:00,400
4,2025-03-01 00:00:00-03:00,2025-03-06 05:00:00-03:00,500
5,2025-03-01 00:00:00-03:00,2025-03-07 06:00:00-03:00,600
6,2025-03-01 00:00:00-03:00,2025-03-08 07:00:00-03:00,700
7,2025-03-01 00:00:00-03:00,2025-03-09 08:00:00-03:00,800


In [96]:
for linha in range(len(fd)):
    dt_end = fd['dt_end'][linha]
    df_filter = dados_candles[dados_candles['open_time'] < dt_end]
    print(len(df_filter))

220
320
420
520
620
720
820
920


In [2]:
df = pd.read_parquet(r"C:\Users\gabri\OneDrive\Documentos\Criptos\Analises\202507\dados_prices_metrics\DADOS_SIMULADOS_TESTE.parquet")
df.head(2)

,Estrategia,Data_Inicio,Data_Fim,Saldo_Final,Lucro_%,Total_Trades,Trades_Lucrativos,Valor_Lucro_Total,Trades_Preuízo,Valor_Preuízo_Total,Lucro_Médio_Trade,Lucro_%_Médio,Prejuízo_Médio_Trade,Prejuízo_%_Médio,Tempo_Simulacao,Moeda,CANDLE_PERIOD
0,UT BOTS,2025-03-01 00:00:00-03:00,2025-03-02 01:00:00-03:00,1014.64921,1.464921,6.0,2.0,16.235549,1.0,-1.586339,8.117775,0.811777,1.586339,0.158634,0.767113,BTC,15m
1,MA RSI e VOLUME,2025-03-01 00:00:00-03:00,2025-03-02 01:00:00-03:00,NaN,NaN,13.0,3.0,3.983190,3.0,-9.585269,1.327730,0.132773,3.195090,0.319509,0.408480,BTC,15m


In [3]:
df['Data_Fim'].unique()

<DatetimeArray>
['2025-03-02 01:00:00-03:00', '2025-03-03 02:00:00-03:00',
 '2025-03-04 03:00:00-03:00', '2025-03-05 04:00:00-03:00',
 '2025-03-06 05:00:00-03:00', '2025-03-07 06:00:00-03:00',
 '2025-03-08 07:00:00-03:00', '2025-03-09 08:00:00-03:00',
 '2025-03-15 07:00:00-03:00', '2025-03-31 07:00:00-03:00']
Length: 10, dtype: datetime64[ns, America/Sao_Paulo]

In [4]:
df['Data_Fim'] = df['Data_Fim'].astype('str')
df['Data_Fim'].unique()

array(['2025-03-02 01:00:00-03:00', '2025-03-03 02:00:00-03:00',
       '2025-03-04 03:00:00-03:00', '2025-03-05 04:00:00-03:00',
       '2025-03-06 05:00:00-03:00', '2025-03-07 06:00:00-03:00',
       '2025-03-08 07:00:00-03:00', '2025-03-09 08:00:00-03:00',
       '2025-03-15 07:00:00-03:00', '2025-03-31 07:00:00-03:00'],
      dtype=object)

In [7]:
df['Data_Fim'] = df['Data_Fim'].map({'2025-03-02 01:00:00-03:00':220,
'2025-03-03 02:00:00-03:00':320,
'2025-03-04 03:00:00-03:00':420,
'2025-03-05 04:00:00-03:00':520,
'2025-03-06 05:00:00-03:00':620,
'2025-03-07 06:00:00-03:00':720,
'2025-03-08 07:00:00-03:00':820,
'2025-03-09 08:00:00-03:00':920,
'2025-03-15 07:00:00-03:00':1376,
'2025-03-31 07:00:00-03:00':2912})

df['Data_Fim'].unique()

array([ 220,  320,  420,  520,  620,  720,  820,  920, 1376, 2912],
      dtype=int64)

In [8]:
print(df[['Data_Fim','Tempo_Simulacao','Estrategia']])

     Data_Fim  Tempo_Simulacao                          Estrategia
0         220         0.767113                             UT BOTS
1         220         0.408480                     MA RSI e VOLUME
2         220         0.305078                MA ANTECIPATION 5_10
3         220         0.283213            MA SIMPLES FALLBACK 5_10
4         220         0.294076                                 RSI
..        ...              ...                                 ...
545      2912       120.485156                               VIDYA
546      2912        10.254935  Volume Weighted Average Price VWAP
547      2912        10.826931                  Williams Alligator
548      2912        86.122756                                 WMA
549      2912         6.146201             Zero Lag Moving Average

[550 rows x 3 columns]


In [12]:
import plotly.express as px
fig = px.scatter(df, 
                 y="Tempo_Simulacao", 
                 x="Data_Fim", 
                 color="Estrategia",
                 title="Dispersão e Linhas por Estratégia",
                 labels={"Tempo_Simulacao": "Tempo de Simulação", "Data_Fim": "Data Fim"},
                 hover_data=["Estrategia"])

# Adiciona linha conectando os pontos por 'Estrategia'
fig.add_traces(px.line(df, y="Tempo_Simulacao", x="Data_Fim", color="Estrategia").data)

fig.show()


In [22]:
df[df['Data_Fim'] == 2912]['Tempo_Simulacao'].describe().round(2)

count     52.00
mean      53.72
std      131.07
min        5.20
25%        8.24
50%       11.04
75%       14.70
max      644.94
Name: Tempo_Simulacao, dtype: float64

In [19]:
df[df['Data_Fim'] == 2912].sort_values(by='Tempo_Simulacao', ascending=False)

,Estrategia,Data_Inicio,Data_Fim,Saldo_Final,Lucro_%,Total_Trades,Trades_Lucrativos,Valor_Lucro_Total,Trades_Preuízo,Valor_Preuízo_Total,Lucro_Médio_Trade,Lucro_%_Médio,Prejuízo_Médio_Trade,Prejuízo_%_Médio,Tempo_Simulacao,Moeda,CANDLE_PERIOD
527,KAMA,2025-03-01 00:00:00-03:00,2912,1034.508457,3.450846,354.0,39.0,427.342496,138.0,-392.834039,10.957500,1.095750,2.846623,0.284662,644.937992,BTC,15m
534,OBV,2025-03-01 00:00:00-03:00,2912,999.487957,-0.051204,536.0,101.0,485.311119,167.0,-485.823162,4.805061,0.480506,2.909121,0.290912,620.324643,BTC,15m
523,Fractals,2025-03-01 00:00:00-03:00,2912,1012.917150,1.291715,128.0,20.0,300.790192,44.0,-287.873042,15.039510,1.503951,6.542569,0.654257,365.448740,BTC,15m
509,ALMA Cruzamento,2025-03-01 00:00:00-03:00,2912,982.568602,-1.743140,184.0,40.0,351.849265,52.0,-369.280662,8.796232,0.879623,7.101551,0.710155,183.053135,BTC,15m
545,VIDYA,2025-03-01 00:00:00-03:00,2912,1031.741113,3.174111,38.0,7.0,133.602864,12.0,-101.861751,19.086123,1.908612,8.488479,0.848848,120.485156,BTC,15m
495,UT BOTS,2025-03-01 00:00:00-03:00,2912,955.392439,-4.460756,182.0,33.0,339.732631,58.0,-384.340192,10.294928,1.029493,6.626555,0.662656,116.413596,BTC,15m
544,Ultimate Oscillator,2025-03-01 00:00:00-03:00,2912,NaN,NaN,5.0,1.0,53.522723,1.0,-48.747083,53.522723,5.352272,48.747083,4.874708,97.918112,BTC,15m
548,WMA,2025-03-01 00:00:00-03:00,2912,1047.818758,4.781876,308.0,43.0,454.775487,111.0,-406.956729,10.576174,1.057617,3.666277,0.366628,86.122756,BTC,15m
532,HULL MOVING AVERAGE,2025-03-01 00:00:00-03:00,2912,1072.031247,7.203125,504.0,111.0,645.800545,141.0,-573.769298,5.818023,0.581802,4.069286,0.406929,67.608156,BTC,15m
508,ALMA,2025-03-01 00:00:00-03:00,2912,1072.257251,7.225725,280.0,50.0,498.007755,90.0,-425.750504,9.960155,0.996016,4.730561,0.473056,63.007999,BTC,15m
